<a href="https://colab.research.google.com/github/typewriter221/Decission-Tree-Classification/blob/master/DecisionTreeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
# https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775

In [ ]:
class DecisionTreeClassifier:
  count=0
  def __init__(self,max_depth):
    self.max_depth = max_depth
  def best_split(self,X,y):
    m = y.size
    if m<2:
      return None,None
    num_parent = [np.sum(y==c)for c in range(self.n_classes_)]
    # Gini of current node.
    best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
    best_idx,best_thr = None, None
    for idx in range(self.n_features_):
      left_classes = [0]*self.n_classes_
      right_classes = num_parent.copy()
      thresholds, classes = zip(*sorted(zip(X[:,idx],y)))
      for i in range(1,m):
        c = classes[i-1]
        left_classes[c]+=1
        right_classes[c]-=1
        
        if thresholds[i-1]==thresholds[i]:
          self.count+=1
          continue
        
        gini_left = 1-sum((left_classes[x]/i)**2 for x in range(self.n_classes_))
        gini_right = 1-sum((right_classes[x]/(m-i))**2 for x in range(self.n_classes_))
        gini_avg = ((i*gini_left)+((m-i)*gini_right))/m
        
        
        if gini_avg<best_gini:
          best_gini = gini_avg
          best_thr = (thresholds[i]+thresholds[i-1])/2
          best_idx = idx
      
    return best_idx,best_thr

  def fit(self,X,y):
    self.n_classes_ = len(set(y))
    self.n_features_ = X.shape[1]
    self.tree_ = self._grow_tree(X,y)
  def _grow_tree(self,X,y,depth=0):

    num_samples_per_class  = [np.sum(y==i) for i in range(self.n_classes_)]
    predicted_class = np.argmax(num_samples_per_class)
    node = Node(
          gini=self._gini(y),
          num_samples=y.size,
          num_samples_per_class=num_samples_per_class,
          predicted_class=predicted_class,
    )
    if depth<self.max_depth:
      idx,thr = self.best_split(X,y)
      if idx is not None:
        indices_left = X[:,idx]<thr
        X_left,Y_left = X[indices_left],y[indices_left]
        X_right,Y_right = X[~indices_left],y[~indices_left]
        node.feature_index = idx
        node.threshold = thr
        node.left = self._grow_tree(X_left,Y_left,depth+1)
        node.right = self._grow_tree(X_right,Y_right,depth+1)

    return node
  def predict(self, X):
    return [self._predict(inputs) for inputs in X]

  def _predict(self, inputs):
    """Predict class for a single sample."""
    node = self.tree_
    print(node.feature_index)
    while node.left:
        if inputs[node.feature_index] < node.threshold:
            node = node.left
        else:
            node = node.right
    return node.predicted_class
  def _gini(self, y):
    """Compute Gini impurity of a non-empty node.
    Gini impurity is defined as Σ p(1-p) over all classes, with p the frequency of a
    class within the node. Since Σ p = 1, this is equivalent to 1 - Σ p^2.
    """
    m = y.size
    return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in range(self.n_classes_))
  def debug(self, feature_names, class_names, show_details=True):
    """Print ASCII visualization of decision tree."""
    self.tree_.debug(feature_names, class_names, show_details)


In [ ]:
class Node:
    """A decision tree node."""

    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

    def debug(self, feature_names, class_names, show_details):
        """Print an ASCII visualization of the tree."""
        lines, _, _, _ = self._debug_aux(
            feature_names, class_names, show_details, root=True
        )
        for line in lines:
            print(line)

    def _debug_aux(self, feature_names, class_names, show_details, root=False):
        # See https://stackoverflow.com/a/54074933/1143396 for similar code.
        is_leaf = not self.right
        if is_leaf:
            lines = [class_names[self.predicted_class]]
        else:
            lines = [
                "{} < {:.2f}".format(feature_names[self.feature_index], self.threshold)
            ]
        if show_details:
            lines += [
                "gini = {:.2f}".format(self.gini),
                "samples = {}".format(self.num_samples),
                str(self.num_samples_per_class),
            ]
        width = max(len(line) for line in lines)
        height = len(lines)
        if is_leaf:
            lines = ["║ {:^{width}} ║".format(line, width=width) for line in lines]
            lines.insert(0, "╔" + "═" * (width + 2) + "╗")
            lines.append("╚" + "═" * (width + 2) + "╝")
        else:
            lines = ["│ {:^{width}} │".format(line, width=width) for line in lines]
            lines.insert(0, "┌" + "─" * (width + 2) + "┐")
            lines.append("└" + "─" * (width + 2) + "┘")
            lines[-2] = "┤" + lines[-2][1:-1] + "├"
        width += 4  # for padding

        if is_leaf:
            middle = width // 2
            lines[0] = lines[0][:middle] + "╧" + lines[0][middle + 1 :]
            return lines, width, height, middle

        # If not a leaf, must have two children.
        left, n, p, x = self.left._debug_aux(feature_names, class_names, show_details)
        right, m, q, y = self.right._debug_aux(feature_names, class_names, show_details)
        top_lines = [n * " " + line + m * " " for line in lines[:-2]]
        # fmt: off
        middle_line = x * " " + "┌" + (n - x - 1) * "─" + lines[-2] + y * "─" + "┐" + (m - y - 1) * " "
        bottom_line = x * " " + "│" + (n - x - 1) * " " + lines[-1] + y * " " + "│" + (m - y - 1) * " "
        # fmt: on
        if p < q:
            left += [n * " "] * (q - p)
        elif q < p:
            right += [m * " "] * (p - q)
        zipped_lines = zip(left, right)
        lines = (
            top_lines
            + [middle_line, bottom_line]
            + [a + width * " " + b for a, b in zipped_lines]
        )
        middle = n + width // 2
        if not root:
            lines[0] = lines[0][:middle] + "┴" + lines[0][middle + 1 :]
        return lines, n + m + width, max(p, q) + 2 + len(top_lines), middle

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/datasets/wifi_localization.txt",delimiter = "\t")
data = df.to_numpy()
X = data[:,:-1]
y = data[:,-1]-1
clf = DecisionTreeClassifier(max_depth=2)
clf.fit(X, y)


# Visualize.
clf.debug(
    feature_names=["Wifi {}".format(i) for i in range(1, 8)],
    class_names=["Room {}".format(i) for i in range(1, 5)],
)


                                                          ┌──────────────────────┐                                                         
                                                          │   Wifi 1 < -54.50    │                                                         
                                                          │     gini = 0.75      │                                                         
                                                          │    samples = 1999    │                                                         
                             ┌────────────────────────────┤ [499, 500, 500, 500] ├─────────────────────────────┐                           
                             │                            └──────────────────────┘                             │                           
                   ┌─────────┴─────────┐                                                             ┌─────────┴────────┐                  
                   │

In [ ]:
clf.count

27266

In [ ]:
inputs = X[-5:-1,:]

clf.predict(inputs)

0
0
0
0


[3, 3, 3, 3]

In [ ]:
inputs = X[0,:]

In [ ]:
inputs[1]

-57

In [ ]:
y[-5:-1]

array([3, 3, 3, 3])